In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Paths to preprocessed 1D CSVs
folders = {
    "Abnormal Heartbeat": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\AHB",
    "Myocardial Infarction": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\MI",
    "Normal": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\NORMAL",
    "History of MI": r"E:\FYP\Cardi 2\Cardiovascular-Detection-using-ECG-images\preprocessed_1d\PM"
}

In [ ]:
all_data = []

for label, folder in folders.items():
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder, file))
            df['Class'] = label
            all_data.append(df)

data = pd.concat(all_data, ignore_index=True)

# Features & labels
X = data.iloc[:, :255].values.astype(np.float32)  # ensure float
y = data['Class'].values

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

# Reshape for 1D CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

num_classes = y_onehot.shape[1]


In [ ]:
inputs = Input(shape=(X_train.shape[1], 1))

# CNN Block
x = Conv1D(128, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(256, kernel_size=5, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)

# Transformer Block
x_norm = LayerNormalization(epsilon=1e-6)(x)
attn_output = MultiHeadAttention(num_heads=8, key_dim=x.shape[2], dropout=0.2)(x_norm, x_norm)
x = Add()([x, attn_output])
x_norm2 = LayerNormalization(epsilon=1e-6)(x)
ff = Dense(256, activation='relu')(x_norm2)
ff = Dropout(0.2)(ff)
x = Add()([x, ff])
x = GlobalAveragePooling1D()(x)

# Fully connected layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('Hybrid_CNN_Transformer_best.h5', monitor='val_accuracy', save_best_only=True)

# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint]
)


In [ ]:
# Predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,6))
plt.imshow(cm, cmap='Blues')
plt.title("Confusion Matrix")
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.xticks(range(num_classes), label_encoder.classes_, rotation=45)
plt.yticks(range(num_classes), label_encoder.classes_)
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()

# Classification Report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


In [ ]:
# Save model and encoder
model.save("Hybrid_CNN_Transformer_final.h5")
import joblib
joblib.dump(label_encoder, "label_encoder_final.pkl")